# get predictions and outcome

In [1]:
'''This script re-fits all non-LSTM models and evaluates them for each cell / dataset.
It also takes a pre-trained LSTM and evaluates it on each cell / dataset.
'''

import os
import sys
sys.path.append('../src')
import data
# from os.path import join as oj
# import sys

# from sklearn.tree import DecisionTreeRegressor
# sys.path.append('../src')
# import numpy as np
# import torch
# import scipy
# from matplotlib import pyplot as plt
# from sklearn import metrics
# import data
# from config import *
# from tqdm import tqdm
# import pickle as pkl
# import train_reg
# from copy import deepcopy
# import config
# import models
# import pandas as pd
# import features
# import outcomes
# import neural_networks
# from sklearn.model_selection import KFold
# from torch import nn, optim
# from torch.nn import functional as F
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# from sklearn.linear_model import LinearRegression, RidgeCV
# from sklearn.svm import SVR
# from collections import defaultdict

In [ ]:
df_train, df_test, feat_names = data.get_snf_mt_vs_wt()
epoch = 10    
feat_name = feat_names[0]
df_train = df_train.dropna()

for df in [df_train, df_test]:
    print(df['mt'].value_counts())


outcome = 'mt'
outcome_def = 'mt'
outcome_binary = 'mt'
                
# print("computing predictions for lstm")                 
models.append('lstm')
checkpoint_fname = f'../models/vps_distingish_mt_vs_wt_epoch={epoch}.pkl'
results = pkl.load(open(checkpoint_fname, 'rb'))
dnn = neural_networks.neural_net_sklearn(
    D_in=40, H=20, p=0, arch='lstm', track_name=feat_name)
dnn.model.load_state_dict(results['model_state_dict'])

df = df_test
X = df[[feat_name]]
y_reg = df[outcome_def] # df['Y_sig_mean_normalized'].values
y = df[outcome_binary].values
#preds = np.logical_and(dnn.predict(X), df['X_max'] > 1500).values.astype(int)  
preds = dnn.predict(X)

# Scatter plot
Show that preds correlates with common features and that erors are where vps may not get recruited.

In [ ]:
print('shapes', y.shape, preds.shape, f'{pd.Series(y).value_counts()=}')

In [ ]:
plt.hist(preds[y==0], label='wt')
plt.hist(preds[y==1], label='mt')
plt.xlabel('Prediction')
plt.ylabel("Count (test)")
plt.legend()
plt.show()

In [ ]:
ys = df['mt'].values

# plt.axhline(0, color='gray', alpha=0.5)
plt.yticks([])
correct = ((preds>0.5) == ys)
err = ~correct
plt.plot(df['Y_sig_mean'][correct], preds[correct], '.', alpha=0.1, markeredgewidth=0) #, c=df['preds'], **kwargs)
plt.plot(df['Y_sig_mean'][err], preds[err], '.', alpha=1, markeredgewidth=0) #, c=df['preds'], **kwargs)
plt.xlabel('Mean fitted vps amplitude')
plt.ylabel('Predicted Prob(mt)')
plt.xscale('log')

# scatter plots

In [ ]:
ys = df['mt'].values
# preds_perc = [percentileofscore(ys, score=p) for p in preds]
preds_perc = preds
# df['preds'] = 

R, C = 2, 3
plt.figure(figsize=(C * 4.5, R * 3))

plt.subplot(R, C, 1)
# plt.axhline(0, color='gray', alpha=0.5)
plt.plot(df['X_max'], preds_perc, '.', alpha=0.1, markeredgewidth=0)
# cs=df['mt'].map({0: 'blue', 1: 'orange'}).values.tolist()) #, c=df['preds'], **kwargs)
plt.xscale('log')
plt.xlabel('Max fitted snf7 amplitude')
plt.ylabel('Predicted Prob(mt)\n')
plt.title('A', loc='left', fontweight='bold')

plt.subplot(R, C, 2)
# plt.axhline(0, color='gray', alpha=0.5)
plt.yticks([])
plt.plot(df['Y_sig_mean'], preds_perc, '.', alpha=0.1, markeredgewidth=0) #, c=df['preds'], **kwargs)
plt.xlabel('Mean fitted vps amplitude')
plt.xscale('log')
plt.title('B', loc='left', fontweight='bold')

plt.subplot(R, C, 3)
# plt.axhline(0, color='gray', alpha=0.5)
plt.plot(df['X_d1'], preds_perc, '.', alpha=0.1, markeredgewidth=0) #, c=df['preds'], **kwargs)
plt.xlabel('DASC (D1)')
plt.yticks([])
plt.title('C', loc='left', fontweight='bold')

plt.subplot(R, C, 4)
# plt.axhline(0, color='gray', alpha=0.5)
plt.plot(df['lifetime'], preds_perc, '.', alpha=0.1, markeredgewidth=0) #, c=df['preds'], **kwargs)
plt.xlabel('Lifetime (sec)')
# plt.ylabel('Predicted auxilin\nsignal strength')
# plt.yticks([])
plt.title('D', loc='left', fontweight='bold')

plt.subplot(R, C, 5)
# plt.axhline(0, color='gray', alpha=0.5)
plt.plot(df['X_peak_idx'], preds_perc, '.', alpha=0.1, markeredgewidth=0) #, c=df['preds'], **kwargs)
plt.xlabel('Time to snf7 peak (sec)')
plt.yticks([])
plt.title('E', loc='left', fontweight='bold')

plt.subplot(R, C, 6)
plt.axhline(0, color='gray', alpha=0.5)
plt.plot(df['mean_total_displacement'], preds_perc, '.', alpha=0.1, markeredgewidth=0) #, c=df['preds'], **kwargs)
plt.xlabel('Mean displacement')
plt.yticks([])
plt.title('F', loc='left', fontweight='bold')

plt.tight_layout()
# viz.savefig('top_feats_scatter')
plt.show()